In [3]:
import os
import json
from typing import List, Dict, Any
from dotenv import load_dotenv

# Import from the proper autogen packages
from autogen import AssistantAgent, UserProxyAgent, config_list_from_json
from autogen.coding import LocalCommandLineCodeExecutor
import autogen

load_dotenv()

# Configure LLM endpoints
config_list = [
    {
        'model': 'gpt-4o',
        'api_key':  os.getenv("OPENAI_API_KEY"),
        'tags': ['tool', 'gpt-4'],
    },
    {
        'model': 'gemini-1.5-pro',
        'api_key': os.getenv("GEMINI_API_KEY"),
        'api_type': 'google',
        'tags': ['tool', 'gemini'],
    },
    {
        'model': 'gemini-1.5-flash',
        'api_key': os.getenv("GEMINI_API_KEY"),
        'api_type': 'google',
        'tags': ['tool', 'gemini'],
    },
    {
        'model': 'gemini-1.0-pro',
        'api_key': os.getenv("GEMINI_API_KEY"),
        'api_type': 'google',
        'tags': ['gemini'],
    },
    {
        "model": "llama3.2:latest",
        "base_url": "http://localhost:11434/v1/",
        "api_key": "ollama",
        "tags": ["local", "ollama"],
    }
]

config_list_file_name = ".config_list"
with open(config_list_file_name, "w") as file:
    json.dump(config_list, file, indent=4)

# Create filtered config lists for different models
gpt_config_list = config_list_from_json(config_list_file_name, filter_dict={"tags": ["gpt-4"]})
gpt_llm_config = {"config_list": gpt_config_list, "timeout": 120}

gemini_config_list = config_list_from_json(config_list_file_name, filter_dict={"tags": ["gemini"]})
gemini_llm_config = {"config_list": gemini_config_list, "timeout": 120}

ollama_config_list = config_list_from_json(config_list_file_name, filter_dict={"tags": ["local"]})
ollama_llm_config = {"config_list": ollama_config_list, "timeout": 500}

# Example arxiv search function
example_f = r"""
def search_arxiv(topic: str, max_results: int = 20) -> List[Dict[Any, Any]]:
    print(f"Searching arXiv for: {topic}")
    client = arxiv.Client()
    search = arxiv.Search(
        query=f"%22{topic}%22",
        max_results=max_results,
        sort_by=arxiv.SortCriterion.SubmittedDate,
        sort_order=arxiv.SortOrder.Descending
    )
    
    results = []
    for paper in client.results(search):
        paper_dict = {
            "title": paper.title,
            "authors": [author.name for author in paper.authors],
            "summary": paper.summary,
            "published": paper.published.strftime("%Y-%m-%d"),
            "url": paper.pdf_url,
            "arxiv_id": paper.get_short_id(),
            "categories": paper.categories
        }
        results.append(paper_dict)
    
    print(f"Found {len(results)} papers on arXiv")
    return results
"""

# Create a UserProxyAgent with code execution configuration
user_proxy = UserProxyAgent(
    name="UserProxyAgent",
    system_message="A human user who needs a technical review document.",
    code_execution_config={
        "last_n_messages": 3,
        "work_dir": "paper_review",
        "use_docker": False
    },
    human_input_mode="NEVER"
)

# Create the researcher agent
researcher = AssistantAgent(
    name="Researcher",
    system_message=f"""You are a researcher specialized in finding relevant research papers.
    You're expert at retrieving and analyzing scientific publications.
    When asked to search for papers for a given topic.
        1. Implement functions for searching publications within recent years from arxiv for the given topic.
        2. You may need to install Python modules i.e. arxiv to proceed the searching.
        3. Here is the code for the search_arxiv function you can modify as required {example_f}.
           Make sure to put "%22" around the query topic as its used to represent quotation for arxiv search for multi word query.
           If search result is not as expected, you should use the trics discussed here, https://info.arxiv.org/help/api/user-manual.html#query_details
        4. Debug and correct your implementation until it returns a non empty valid list of publication 
            from arxiv for the given query. Make sure that the returned publications matches the topic.
            if not, find the bug and solve it. 
        5. There should be a introduction and also a summary section in the document.
        6. After retrieving, format your findings as a structured report with clear sections and paper summaries.
        7. After retrieving, read all the papers summary and prepare the summary of all the publication works 
            within 15 sentences and append at the document. You can cite the papers in the summary text as IEEE bibliography style.
        8. The final document should be in markdown format.
        9. If there is no more task pending at your side, return "TERMINATE" otherwise return "CONTINUE".
    """,
    llm_config=gemini_llm_config,
    code_execution_config={
        "last_n_messages": 3,
        "executor": LocalCommandLineCodeExecutor(work_dir="paper_review")
    }
)

def run_review_workflow(topic: str) -> str:
    # Create group chat
    groupchat = autogen.GroupChat(
        agents=[user_proxy, researcher],
        messages=[],
        max_round=20,
        speaker_selection_method="round_robin",
        allow_repeat_speaker=True
    )
    
    # Create manager
    manager = autogen.GroupChatManager(
        groupchat=groupchat,
        llm_config=gemini_llm_config
    )
    
    # Initiate chat
    user_proxy.initiate_chat(
        manager,
        message=f"""
        1. You need to generate a comprehensive technical review document on "{topic}". 
        2. Ask Researcher to search the topic on arxiv and prepare the technical review document.
        2. After you get the final technical review document, implement create_pdf function which can convert the review document to pdf. 
        3. You may need to install dependent Python module i.e. markdown2, weasyprint and so on. to implement and debug your function until it returns valid PDF.
           Provide the the final review document retrieved from the Writer to the create_pdf function. 
           File name should be "{topic.replace(' ', '_')}_review.pdf"
        4. The final output should be a PDF file saved in the working directory.
        5. If there is no more task pending at your side, return "TERMINATE" otherwise return "CONTINUE".
        """,
        is_termination_msg=lambda x: x.get("content", "").rstrip().endswith("TERMINATE"),
        human_input_mode="NEVER",
    )
    
    return f"{topic.replace(' ', '_')}_review.pdf"

if __name__ == "__main__":
    topic = "BCG Signal Processing"
    pdf_path = run_review_workflow(topic)
    print(f"Workflow completed. Review document should be at: {pdf_path}")

UserProxyAgent (to chat_manager):


        1. You need to generate a comprehensive technical review document on "BCG Signal Processing". 
        2. Ask Researcher to search the topic on arxiv and prepare the technical review document.
        2. After you get the final technical review document, implement create_pdf function which can convert the review document to pdf. 
        3. You may need to install dependent Python module i.e. markdown2, weasyprint and so on. to implement and debug your function until it returns valid PDF.
           Provide the the final review document retrieved from the Writer to the create_pdf function. 
           File name should be "BCG_Signal_Processing_review.pdf"
        4. The final output should be a PDF file saved in the working directory.
        5. If there is no more task pending at your side, return "TERMINATE" otherwise return "CONTINUE".
        

--------------------------------------------------------------------------------

Next speaker:

ClientError: 429 RESOURCE_EXHAUSTED. {'error': {'code': 429, 'message': 'Resource has been exhausted (e.g. check quota).', 'status': 'RESOURCE_EXHAUSTED'}}